In [4]:
import torch
import torch.nn.functional as F

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [6]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
HF_HUB_DISABLE_SYMLINKS_WARNING=False
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained('t5-base')

TypeError: 'NoneType' object is not callable

In [ ]:
input_text = 'summarize: Hinsichtlich eines absoluten Niedergangs ist zu sagen, dass die USA zwar sehr reale Probleme haben, aber die amerikanische Wirtschaft dennoch hoch produktiv bleibt. Amerika liegt an erster Stelle bei den Gesamtausgaben für F&amp;E, Hochschulrankings, Nobelpreisen und auch bei Unternehmensindizes. Laut Angaben des Weltwirtschaftsforums, das letzten Monat seinen Bereicht über wirtschaftliche Wettbewerbsfähigkeit veröffentlichte, liegen die USA auf Platz fünf der wettbewerbsfähigsten Ökonomien der Welt (hinter den kleinen Volkswirtschaften der Schweiz, Schwedens, Finnlands und Singapurs). China rangiert erst auf Platz 26.'
input_ids = tokenizer.encode(input_text, return_tensors="pt")

In [4]:
output_ids = model.generate(input_ids)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

C:\Users\shepa\anaconda3\lib\site-packages\transformers\generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [5]:
output_text

'Angaben des Weltwirtschaftsforums: die USA sind auf Platz fünf der wettbewerbs'

In [6]:
with open("train.labeled", "r", encoding="utf-8") as f:
    text = f.read()

In [7]:
sentences = text.split("\n\n")
splitted = [sentence.split("\nEnglish:\n") for sentence in sentences]
german_sents = [sentence[0][8:] for sentence in splitted if sentence[0] != ""]
english_sents = [sentence[1] for sentence in splitted if sentence[0] != ""]

In [8]:
from torch.optim import AdamW
model.train()
optimizer = AdamW(model.parameters(), lr=1e-4)

In [9]:
model.eval()
output_ids = model.generate(input_ids)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
output_text

'Angaben des Weltwirtschaftsforums: die USA sind auf Platz fünf der wettbewerbs'

In [ ]:
for i in range(10000):
    model.train()
    input_ids = tokenizer(f"translate German to English: {german_sents[i]}", return_tensors="pt").input_ids
    labels = tokenizer(english_sents[i], return_tensors="pt").input_ids

    # the forward function automatically creates the correct decoder_input_ids
    loss = model(input_ids=input_ids, labels=labels).loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad(set_to_none=True)
    
    model.eval()
    output_ids = model.generate(input_ids, max_length=100000)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    if i % 10 == 0:
        print(output_text)

Die Wirtschaftskrise scheint die naheliegendste Erklärung zu sein, vielleicht zu naheliegend.


In [ ]:
# Suppose we have the following 2 training examples:
input_sequence_1 = "Welcome to NYC"
output_sequence_1 = "Bienvenue à NYC"

input_sequence_2 = "HuggingFace is a company"
output_sequence_2 = "HuggingFace est une entreprise"

# encode the inputs
task_prefix = "translate German to English: "
input_sequences = [input_sequence_1, input_sequence_2]

encoding = tokenizer(
    [task_prefix + sequence for sequence in input_sequences],
    padding="longest",
    max_length=max_source_length,
    truncation=True,
    return_tensors="pt",
)

input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

# encode the targets
target_encoding = tokenizer(
    [output_sequence_1, output_sequence_2],
    padding="longest",
    max_length=max_target_length,
    truncation=True,
    return_tensors="pt",
)
labels = target_encoding.input_ids

# replace padding token id's of the labels by -100 so it's ignored by the loss
labels[labels == tokenizer.pad_token_id] = -100

# forward pass
loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss
loss.item()